In [1]:
from bokeh.io import output_file, show, output_notebook,curdoc
from bokeh.layouts import widgetbox

from bokeh.models import ColumnDataSource, Slider
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn, Dropdown
from bokeh.embed import components
from datetime import date
from random import randint
from bokeh.plotting import figure
from bokeh.sampledata.sea_surface_temperature import sea_surface_temperature
from bokeh.layouts import column, row
from bokeh.themes import Theme
import yaml


In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
import pandas as pd
df = pd.read_csv('../data/headcount.csv', sep = '\t')
df.head()

,FT or PT,Month,Headcount
0,PT,Jun-21,902
1,FT,Jun-21,7510
2,PT,May-21,905
3,FT,May-21,7659
4,PT,Apr-21,906


In [4]:

def create_data_table(df):
        
    data = dict(df)
    source = ColumnDataSource(data)

    columns = [
            TableColumn(field="Month", title="Month"),
            TableColumn(field="FT or PT", title="FT or PT"),
            TableColumn(field="Headcount", title="Headcount"),
        
        ]
    data_table = DataTable(source=source, columns=columns, width=600, height=400)
    return data_table




In [5]:
df2 = df.groupby(['Month']).sum().reset_index()
df2.head()

,Month,Headcount
0,Apr-21,8633
1,Aug-20,10835
2,Dec-20,9915
3,Feb-21,9397
4,Jan-21,9793


In [6]:
def bar_chart(df):
    # prepare some data
    month = list(df2['Month'].values)
    headcount = list(df2['Headcount'].values)

    # output to static HTML file

    # create a new plot with a title and axis labels
    source = ColumnDataSource(data=dict(df))
    p2 = figure(title="simple line example", x_range=month,)#)x_axis_label='month', y_axis_label='headcount')
    p2.sizing_mode = 'stretch_width'
    # add a line renderer with legend and line thickness
    p2.vbar(x = 'Month', top = 'Headcount', width =  0.5, legend_label="Temp.", line_width=2, color = 'blue',source = source)
    #p2.line(x, y, legend_label="Temp.", line_width=2,color = 'blue' )
    #p2.line(x, y, legend_label="Temp.", line_width=2,color = 'orange')


    # show the results
    show(p2,)

In [7]:
bar_chart(df)

In [8]:
def modify_doc(doc):
    df = sea_surface_temperature.copy()
    source = ColumnDataSource(data=df)

    plot = figure(x_axis_type='datetime', y_range=(0, 25),
                  y_axis_label='Temperature (Celsius)',
                  title="Sea Surface Temperature at 43.18, -70.43")
    plot.line('time', 'temperature', source=source)

    def callback(attr, old, new):
        if new == 0:
            data = df
        else:
            data = df.rolling('{0}D'.format(new)).mean()
        source.data = dict(ColumnDataSource(data=data).data)

    slider = Slider(start=0, end=30, value=0, step=2, title="Smoothing by N Days")
    slider.on_change('value', callback)
    
    columns = [
            TableColumn(field="time", title="time"),
            TableColumn(field="temperature", title="temperature"),
        
        ]
    data_table = DataTable(source=source, columns=columns, width=600, height=400)
    doc.add_root(row(slider, plot, data_table))
    return doc
    #doc.theme = Theme(json=yaml.load(
    """
        attrs:
            Figure:
                background_fill_color: "#DDDDDD"
                outline_line_color: white
                toolbar_location: above
                height: 500
                width: 800
            Grid:
                grid_line_dash: [6, 4]
                grid_line_color: white
    
    """
    #))"""

    
    
    
def generate_app(doc):

    df = pd.read_csv('../data/headcount.csv', sep = '\t')

    df = pd.read_csv("example.csv", index_col=0)
    data_source = ColumnDataSource(df)
    line_plot1 = figure(sizing_mode="stretch_both")
    line_plot1.line(
        source=data_source,
        x="x",
        y="y",
    )
    line_plot2 = figure(sizing_mode="stretch_both")
    line_plot2.line(
        source=data_source,
        x="y",
        y="x",
    )
    sliders = [
        Slider(start=0, end=5, value=5, step=1, title="Max X"),
        Slider(start=0, end=25, value=25, step=1, title="Max Y"),
    ]

    def update_graph(attr, old, new):
        x_max = sliders[0].value
        y_max = sliders[1].value
        filtered_df = df[(df["x"] <= x_max) & (df["y"] <= y_max)]
        new_data = ColumnDataSource(filtered_df).data
        data_source.data = dict(new_data)

    for slider in sliders:
        slider.on_change("value", update_graph)

    layout = grid(
        [
            [
                line_plot1,
                column(sliders),
            ],
            line_plot2,
        ],
        sizing_mode="stretch_both",
    )

    doc.add_root(layout)

In [9]:

show(modify_doc)


# Headcount

In [14]:
headcount_df = pd.read_csv('../data/headcount.txt',sep='\t', thousands=',')
headcount_df['Date'] =  pd.to_datetime({'year':headcount_df.CAL_YEAR,
                                  'month': headcount_df.CAL_MONTH,
                                  'day':1})
headcount_df.head()

D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,CAL_YEAR,CAL_MONTH,HOME_LOCATION,JOB_DESC,STEP,GL_ACCT_ID,COST_CNTR_ID,FULL_OR_PART_TIME,AGE_GROUP,HEADCOUNT,Date
0,2019.0,1.0,SAT,Ramp Agent,11.0,4263100,0000014563,F,56 - 60,13.0,2019-01-01
1,2019.0,1.0,PVD,OPS Agent,11.0,4261100,0000014525,F,46 - 50,4.0,2019-01-01
2,2019.0,1.0,STL,Provisioning Agent,4.0,4265100,0000016027,F,46 - 50,1.0,2019-01-01
3,2019.0,1.0,TPA,Ramp Agent,9.0,4263100,0000014533,F,51 - 55,1.0,2019-01-01
4,2019.0,1.0,HOU,Ramp Agent,3.0,4263100,0000014526,P,31 - 35,1.0,2019-01-01


In [16]:
headcount_df.pivot_table(values= 'HEADCOUNT',index = [ 'FULL_OR_PART_TIME'],
                        columns = 'Date', aggfunc='sum'
                ).reset_index()

Date,FULL_OR_PART_TIME,2019-01-01 00:00:00,2019-02-01 00:00:00,2019-03-01 00:00:00,2019-04-01 00:00:00,2019-05-01 00:00:00,2019-06-01 00:00:00,2019-07-01 00:00:00,2019-08-01 00:00:00,2019-09-01 00:00:00,...,2019-11-01 00:00:00,2019-12-01 00:00:00,2020-01-01 00:00:00,2020-02-01 00:00:00,2020-03-01 00:00:00,2020-04-01 00:00:00,2020-05-01 00:00:00,2020-06-01 00:00:00,2020-07-01 00:00:00,2020-08-01 00:00:00
0,F,20293.0,20315.0,20332.0,20366.0,20433.0,20495.0,20489.0,20644.0,20651.0,...,20810.0,20857.0,20865.0,20980.0,20973.0,20959.0,20917.0,20874.0,20824.0,20735.0
1,P,1980.0,1943.0,1894.0,1895.0,1875.0,1874.0,1853.0,1891.0,1902.0,...,1956.0,1957.0,1968.0,1990.0,2028.0,2012.0,2008.0,1996.0,1984.0,1972.0


In [11]:
headcount_df = df.pivot_table(values = 'Headcount', index = ['FT or PT'],columns='Month').reset_index()
headcount_df.head()

Month,FT or PT,Apr-21,Aug-20,Dec-20,Feb-21,Jan-21,Jul-20,Jun-21,Mar-21,May-21,Nov-20,Oct-20,Sep-20
0,FT,7727,9817,8926,8430,8805,9970,7510,8258,7659,9186,9396,9622
1,PT,906,1018,989,967,988,1020,902,960,905,997,1007,1016


In [18]:
headcount_df.values 

array([[2019.0, 1.0, 'SAT', ..., '56 - 60', 13.0,
        Timestamp('2019-01-01 00:00:00')],
       [2019.0, 1.0, 'PVD', ..., '46 - 50', 4.0,
        Timestamp('2019-01-01 00:00:00')],
       [2019.0, 1.0, 'STL', ..., '46 - 50', 1.0,
        Timestamp('2019-01-01 00:00:00')],
       ...,
       [2020.0, 8.0, 'PHX', ..., '36 - 40', 1.0,
        Timestamp('2020-08-01 00:00:00')],
       [2020.0, 8.0, 'GEG', ..., '46 - 50', 1.0,
        Timestamp('2020-08-01 00:00:00')],
       [2020.0, 8.0, 'ORF', ..., '61 - 65', 1.0,
        Timestamp('2020-08-01 00:00:00')]], dtype=object)